# Loading the 10,000 Song Subset to Neo4j

*Andrea Soto*  
*MIDS W205 Final Project*  
*Project Name: Graph Model of the Million Song Dataset*

---

# Notebook Overview

This notebook is a continuation of the notebook [Step 2 - Process Subset.ipynb](./Step 2 - Process Subset.ipynb) were the CSV files containing the graph nodes and relationships were created. **The goal of this notebook is to load the subset dataset with 10,000 songs into Neo4j.** The code developed was then compiled in scripts that were used to process the full data.

The tables below summarize the files that contain the graph data. These files are located in the directory **MillionSongSubset/graph**.

**Nodes**

|No.|Node Label|File Name| Format |
|:--:|:--|:--|:--|
|1|Artists|nodes_artists.csv| 'artist_id', 'artist_mbid', 'artist_7did', 'artist_name'|
|2|Songs|nodes_songs.csv|'song_id', 'track_id', 'title', 'dance', 'dur', 'energy','loudness'|
|3|Albums|nodes_albums.csv| 'album_name'|
|4|Year|nodes_years.csv| 'year'|
|5|Tags|nodes_tags.csv| 'tag'|

**Relationships**

|No.|Relationship Structure|File Name| Format |
|:--:|:--|:--|:--|
|1|(ARTIST) - [SIMILAR_TO] -> (ARTIST)|rel_similar_artists.csv|'from_artist_id', 'to_artist_id'|
|2|(ARTIST) - [PERFORMS] -> (SONG)|rel_performs.csv|'artist_id', 'song_id'|
|3|(ARTIST) - [HAS_ALBUM] -> (ALBUM)|rel_artist_has_album.csv|'artist_id', 'album_name'|
|4|(ARTIST) - [HAS_TAG] -> (TAG)|rel_artist_has_tag.csv|'artist_id', 'tag_name', 'normalized_frq', 'normalized_weight'|
|5|(SONG) - [IN_ALBUM] -> (ALBUM)|rel_song_in_album.csv|'song_id', 'album_name'|
|6|(SONG) - [SIMILAR_TO] -> (SONG)| rel_similar_songs.csv|'from_song_id', 'to_song_id', 'similarity_weight'|
|7|(SONG) - [HAS_TAG] -> (TAG)| rel_song_has_tag.csv|'from_song_id', 'to_song_id', 'normalized_weight'|
|8|(SONG) - [RELEASED_ON] -> (YEAR)| rel_song_year.csv|'song_id', 'year'|

### Load Python Libraries

In [2]:
from py2neo import Graph
from py2neo import authenticate
import os

path = 'file:' + os.getcwd() + '/MillionSongSubset/graph/'

### Check that requiered enviroment names exist

In [3]:
os.environ['NEO4J_HOME']

'/graph/neo4j/bin'

In [4]:
os.environ['INSTANCE_PDNS']

'ec2-54-91-193-97.compute-1.amazonaws.com'

## Start Neo4j Server

Assumes the environment variable NEO4J_HOME exists and points to [installation_path]/neo4j/bin

In [7]:
!$NEO4J_HOME/neo4j start

Starting Neo4j Server...WARNING: not changing user
process [27766]... waiting for server to be ready....... OK.
http://localhost:7474/ is ready.


### Connect to Neo4j

In [5]:
pdns = os.environ['INSTANCE_PDNS']

# Authenticate
authenticate(pdns+":7474", "neo4j", "redpill")

# connect to authenticated graph database
graph = Graph("http://"+pdns+":7474/db/data/")

In [24]:
from py2neo.packages.httpstream import http
http.socket_timeout = 3600

### Create unique constraints and index for 'track_id' property of SONG

In [27]:
# List of constraints to create (if not in place already)
unique_constraints = [('ARTIST', 'id'),('SONG','id'),('ALBUM','name'),('TAG','tag'),('YEAR','year')]

for label,prop in unique_constraints:
    try:
        # Create constraint
        graph.schema.create_uniqueness_constraint(label, prop)
    except:
        pass

# Create index
try:
    graph.schema.create_index('SONG','trackid')
except:
    pass

---
# Load Nodes

### 1. Artists

In [10]:
%%time
# Artists
csv_path = path + 'nodes_artists.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MERGE (a:ARTIST {id:line.artist_id}) '
st += 'ON CREATE SET a.idmb=line.artist_mbid, a.id7d=line.artist_7did, a.name=lower(line.artist_name);'

tx.append(st)
tx.commit()

CPU times: user 3 ms, sys: 0 ns, total: 3 ms
Wall time: 4.59 s


### 2. Songs

In [11]:
%%time
# Songs
csv_path = path + 'nodes_songs.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MERGE (s:SONG {id:line.song_id}) '
st += 'ON CREATE SET s.trackid=line.track_id, s.title=lower(line.song_name),'
st += 's.danceability=TOFLOAT(line.danceability),s.duration = TOFLOAT(line.duration),'
st += 's.energy= TOFLOAT(line.energy), s.loudness = TOFLOAT(line.loudness);'

tx.append(st)
tx.commit()

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 3.34 s


### 3. Albums

In [12]:
%%time
# Albums
csv_path = path + 'nodes_albums.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MERGE (a:ALBUM {name: lower(line.album_name)});'

tx.append(st)
tx.commit()

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 1.15 s


### 4. Years

In [13]:
%%time
# Year
csv_path = path + 'nodes_years.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MERGE (y:YEAR {year: TOINT(line.year)});'

tx.append(st)
tx.commit()

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 171 ms


### 5. Tags

In [14]:
%%time
# Tags
csv_path = path + 'nodes_tags.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MERGE (:TAG {tag: lower(line.tag_name)});'

tx.append(st)
tx.commit()

CPU times: user 2 ms, sys: 1 ms, total: 3 ms
Wall time: 4.21 s


# Relationships

### 1. Artist - SIMILAR_TO -> Artist

In [17]:
%time
# (ARTIST)-[SIMILAR_TO]-> (ARTIST)
csv_path = path + 'rel_similar_artists.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MATCH (from:ARTIST {id:line.from_artist}) '
st += 'MATCH (to:ARTIST {id:line.to_artist}) '
st += 'MERGE (from)-[:SIMILAR_TO]->(to)'

tx.append(st)
tx.commit()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.9 µs


### 2. Artist - PERFORMS -> Song

In [18]:
%%time
# (ARTIST)-[PERFORMS]-> (SONG)
csv_path = path + 'rel_performs.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MATCH (a:ARTIST {id:line.artist_id}) '
st += 'MATCH (s:SONG {id:line.song_id}) '
st += 'MERGE (a)-[:PERFORMS]->(s)'

tx.append(st)
tx.commit()

CPU times: user 3 ms, sys: 999 µs, total: 4 ms
Wall time: 1.23 s


### 3. Artist - HAS_ALBUM -> Album

In [19]:
%%time
# (ARTIST)-[HAS_ALBUM]-> (ALBUM)
csv_path = path + 'rel_artist_has_album.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MATCH (a:ARTIST {id:line.artist_id}) '
st += 'MATCH (m:ALBUM {name:lower(line.album_name)}) '
st += 'MERGE (a)-[:HAS_ALBUM]->(m);'

tx.append(st)
tx.commit()

CPU times: user 2 ms, sys: 1 ms, total: 3 ms
Wall time: 1.1 s


### 4. Artist - HAS_TAG -> Tag

In [26]:
%%time
# (ARTIST)-[HAS_TAG]-> (TAG)
csv_path = path + 'rel_artist_has_tag.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MATCH (a:ARTIST {id:line.artist_id}) '
st += 'MATCH (t:TAG {tag:lower(line.tag_name)}) '
st += 'MERGE (a)-[r:HAS_TAG]->(t)'
st += 'ON CREATE SET r.frq=TOFLOAT(line.tag_frq), r.weight=TOFLOAT(line.tag_w);'

tx.append(st)
tx.commit()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 15.8 s


### 5. Song - IN_ALBUM -> Album

In [20]:
%%time
# (SONG)-[IN_ALBUM]-> (ALBUM)
csv_path = path + 'rel_song_in_album.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MATCH (s:SONG {id:line.song_id}) '
st += 'MATCH (a:ALBUM {name:lower(line.album_name)}) '
st += 'MERGE (s)-[:IN_ALBUM]->(a);'

tx.append(st)
tx.commit()

CPU times: user 2 ms, sys: 1 ms, total: 3 ms
Wall time: 1.31 s


### 6. Song - SIMILAR_TO -> Song

In [34]:
path + 'rel_similar_songs.csv'

'file:/data/asoto/projectW205/MillionSongSubset/graph/rel_similar_songs.csv'

In [35]:
!pwd

/data/asoto/projectW205


In [36]:
!head MillionSongSubset/graph/rel_similar_songs.csv

from_track,to_track,sim_measure
TRBBOPX12903D106F7,TRNTHZA12903D106FA,1
TRBBOPX12903D106F7,TRWXSDH12903D106F2,0.97073
TRBBOPX12903D106F7,TREOIQG12903CC9DE1,0.0727542
TRBBOPX12903D106F7,TRYJWDG128F930AA09,0.0714807
TRBBOPX12903D106F7,TROQSRY128F930E0D5,0.0546577
TRBBOPX12903D106F7,TRPNFCQ128F930AA02,0.0542935
TRBBOPX12903D106F7,TRDLDXL12903CFFF83,0.0537399
TRBBOPX12903D106F7,TRYNNVL12903CFFF78,0.0536009
TRBBOPX12903D106F7,TRBLAPG128F4244368,0.0534808


In [38]:
%time
# (SONG)-[SIMILAR_TO]-> (SONG)
csv_path = path + 'rel_similar_songs.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MATCH (from:SONG {trackid:line.from_track}) '
st += 'MATCH (to:SONG {trackid:line.to_track}) '
st += 'MERGE (from)-[r:SIMILAR_TO]->(to)'
st += 'ON CREATE SET r.weight=TOFLOAT(line.sim_measure);'

tx.append(st)
tx.commit()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


### 7. Song - HAS_TAG -> Tag

In [37]:
%%time
# (SONG)-[HAS_TAG]-> (TAG)
csv_path = path + 'rel_song_has_tag.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MATCH (s:SONG {trackid:line.track_id}) '
st += 'MATCH (t:TAG {tag:lower(line.tag_name)}) '
st += 'MERGE (s)-[r:HAS_TAG]->(t)'
st += 'ON CREATE SET r.weight=TOFLOAT(line.tag_w);'

tx.append(st)
tx.commit()

CPU times: user 2 ms, sys: 4 ms, total: 6 ms
Wall time: 14.6 s


### 8. Song - RELEASED_ON -> Tag

In [32]:
%%time
# (SONG)-[RELEASED_ON]-> (YEAR)
csv_path = path + 'rel_song_year.csv'

tx = graph.cypher.begin()

st = 'USING PERIODIC COMMIT 1000 '
st += 'LOAD CSV WITH HEADERS FROM "' + csv_path + '" AS line '
st += 'MATCH (s:SONG {id:line.song_id}) '
st += 'MATCH (y:YEAR {year:TOINT(line.year)}) '
st += 'MERGE (s)-[:RELEASED_ON]->(y);'

tx.append(st)
tx.commit()

CPU times: user 1 ms, sys: 1 ms, total: 2 ms
Wall time: 942 ms


---
# Quick queries to confirm data was loaded correctly

### Nodes

In [41]:
tx = graph.cypher.begin()

st = 'MATCH n RETURN DISTINCT LABELS(n), count(n)'

tx.append(st)
tx.commit()

   | LABELS(n)   | count(n)
---+-------------+----------
 1 | [u'ALBUM']  |     7823
 2 | [u'TAG']    |    35112
 3 | [u'SONG']   |    10000
 4 | [u'YEAR']   |       69
 5 | [u'ARTIST'] |     3888


### Relationships

In [42]:
tx = graph.cypher.begin()

st = 'MATCH (a)-[r]->(b) '
st += 'WHERE labels(a) <> [] AND labels(b) <> [] '
st += 'RETURN DISTINCT head(labels(a)) AS This, type(r) as To, head(labels(b)) AS That, count(r) '
st += 'LIMIT 10 '

tx.append(st)
tx.commit()

   | This   | To          | That   | count(r)
---+--------+-------------+--------+----------
 1 | SONG   | IN_ALBUM    | ALBUM  |    10000
 2 | SONG   | SIMILAR_TO  | SONG   |     9215
 3 | SONG   | RELEASED_ON | YEAR   |     4680
 4 | ARTIST | HAS_ALBUM   | ALBUM  |     8061
 5 | ARTIST | HAS_TAG     | TAG    |    96825
 6 | SONG   | HAS_TAG     | TAG    |    99296
 7 | ARTIST | PERFORMS    | SONG   |    10000
 8 | ARTIST | SIMILAR_TO  | ARTIST |    42970


### Sample Query

In [ ]:
MATCH p=(:ARTIST)-->(s:SONG)-->(:SONG)<--(:ARTIST)
WHERE s.title = "don't stop the music"
RETURN p

![Sample Query](./images/Query_DontStopTheMusic.png)

---
# Final Scripts

The notebook [Step 4 - Process Entire Dataset.ipynb](./Step 4 - Process Entire Dataset.ipynb) has the final scripts developed to process the entire Million Song Dataset and a description of how to run the scripts.